In [ ]:
# Import some libraries
import os
import numpy as np
import mne
import glob
import os
%matplotlib qt 

data_directory = 'C:/Users/mvmigem/Documents/data/project_1/localiser_dat/'
dir_list = glob.glob(data_directory+'*')


In [ ]:

data_directory = 'C:/Users/mvmigem/Documents/data/project_1/localiser_dat/'
dir_list = glob.glob(data_directory+'*')
epochs = []
evokeds = []

for i in range(len(dir_list)):


    raw = mne.io.read_raw_bdf(dir_list[i], preload = True)

    # Rename and adress channels

    fix_chans = {'EXG1':'eye_above','EXG2':'eye_below',
                'EXG3':'eye_left','EXG4':'eye_right',
                'EXG5':'M1','EXG6':'M2'}
    raw.rename_channels(fix_chans)

    # we still have two exg channels which weren't actually recorded though (EXG7
    # and EXG8) these are empty, so we'll drop them
    raw.drop_channels(['EXG7', 'EXG8'])
    print(raw.info['ch_names'])

    # we'll also reset the channel types, so MNE knows what is 'brain' data
    raw.set_channel_types({'M1':'eeg', 'M2':'eeg',
                        'eye_above':'eog', 'eye_below':'eog',
                        'eye_left':'eog', 'eye_right': 'eog'})

    print(raw.info)

    # Rereference to mastoids
    raw.set_eeg_reference(ref_channels = ['M1','M2'])
    # then drop them
    raw.drop_channels(['M1','M2'])

    # Select montage
    montage = mne.channels.make_standard_montage('biosemi64')

    # There is a mismatch between the names of the recording and the names of the montage
    # This dict is to rename the channel names to fit the montage
    mon_chnames = montage.ch_names
    raw_chnames = raw.info['ch_names']
    rename_channels = dict(zip(raw_chnames[:64],mon_chnames))
    raw.rename_channels(rename_channels)

    # Set montage
    raw.set_montage(montage)

    # Downsampling variables (logic -> https://mne.tools/stable/auto_tutorials/preprocessing/30_filtering_resampling.html#best-practices)
    current_sfreq = raw.info['sfreq']
    desired_sfreq = 256  # Hz
    decim = np.round(current_sfreq / desired_sfreq).astype(int)
    obtained_sfreq = current_sfreq / decim
    lowpass_freq = obtained_sfreq / 3.


    raw_filtered = raw.copy().notch_filter(freqs = 50, fir_design = 'firwin', verbose=None,n_jobs=-1)
    raw_filtered = raw_filtered.copy().filter(l_freq=0.1, h_freq=lowpass_freq,n_jobs=-1)


    # Plot to reject bad channels manually
    raw_filtered.compute_psd().plot()
    raw_filtered.plot(n_channels=64, block = True)
    # Then intepolate bad channels
    interp_filt_raw = raw_filtered.copy().interpolate_bads(reset_bads = True)

    # Annotate events
    events = mne.find_events(interp_filt_raw)
    # Event dict
    event_id = {        # This needs to be short in the online preprocess only 4 trigger markers (4 quads)
        'position1':80,'position2':81, 'position3':82,'position4':83, 
    }

    # Eye artifact rejection
    almost_clean = interp_filt_raw.copy()

    # we can also identify eog events algorithmically via "find_eog_events" this
    # produces a list of 'events' around each blink (hopefully). This applies a 
    # filter and then identifies peaks in the eog to find likely blinks. We can 
    # adjust the threshold, via thresh. but default should be okay for now.
    eog_events = mne.preprocessing.find_eog_events(almost_clean)
    # we'll say that the blinks start a tiny bit earlier than 
    onsets = eog_events[:, 0] / almost_clean.info["sfreq"] - 0.25
    # we'll assume they're all half a second long
    dur = [0.5] * len(eog_events)
    descriptions = ["bad blink"] * len(eog_events)
    blink_annot = mne.Annotations(onsets,
                                dur,
                                descriptions,
                                orig_time = almost_clean.info["meas_date"])
    almost_clean.set_annotations(blink_annot)

    # Epoch the data
    epochs_stimlock = mne.Epochs(almost_clean, events, event_id = event_id,
        tmin = -0.1, tmax = 0.5, proj = False, baseline = (None,0), decim=decim, #from previous cell
        detrend = None, verbose = True, reject_by_annotation= True, preload = True)

    epochs.append(epochs_stimlock)

    evoked_pos1 = epochs_stimlock['position1'].average()
    evoked_pos2 = epochs_stimlock['position2'].average()
    evoked_pos3 = epochs_stimlock['position3'].average()
    evoked_pos4 = epochs_stimlock['position4'].average()

    evoked = [evoked_pos1, evoked_pos2, evoked_pos3, evoked_pos4]
    evokeds.append(evoked)
